# Overview

This notebook illustrates how to use Prefect to schedule and keep track of tasks running at NERSC.

In [1]:
from sfapi_client import Client
from sfapi_client.compute import Machine
import prefect
from pathlib import Path

In [2]:
# Create Perlmutter job scripts

def random_number_job_script( random_seed ):
    """
    Creates a job script that runs on Perlmutter,
    and generates a random number and saves it in a file `sfapi_demo_{seed}.txt`
    """
    import random
    random.seed(random_seed)
    
    script = f"""#!/bin/bash
#SBATCH -C cpu
#SBATCH -q shared
#SBATCH -N 1
#SBATCH -c 1
#SBATCH -t 1
#SBATCH -o /global/u2/r/rlehe/sfapi_demo_{random_seed}.txt

echo "Completed run {random.randint(1, 100)}"
"""
    return script
    
list_script = f"""#!/bin/bash
#SBATCH -C cpu
#SBATCH -q shared
#SBATCH -N 1
#SBATCH -c 1
#SBATCH -t 1
#SBATCH -o /global/u2/r/rlehe/list.txt

cat sfapi_demo*
"""

In [3]:
# Connect to Perlmutter with the SFAPI
key = Path("./priv_key.pem")
client = Client(key=key)
perlmutter = client.compute(Machine.perlmutter)

@prefect.task
def launch_script_on_perlmutter( script ):
    """
    Launches a script on Perlmutter using the SFAPI,
    and track it with Prefect (using task)
    """
    # Launch a job with the SFAPI and wait until it finishes
    sfapi_job = perlmutter.submit_job( script )
    # Blocking command that waits for the job to complete 
    sfapi_job.complete()

@prefect.flow
def run_jobs_serial():
    launch_script_on_perlmutter( random_number_job_script(0) )
    launch_script_on_perlmutter( random_number_job_script(3) )
    launch_script_on_perlmutter( random_number_job_script(7) )
    launch_script_on_perlmutter( list_script )

In [5]:
# Launch the job and track it with Prefect
run_jobs_serial()

11:36:55.464 | INFO    | prefect.engine - Created flow run 'magenta-camel' for flow 'run-jobs'

11:36:55.473 | INFO    | Flow run 'magenta-camel' - View at https://app.prefect.cloud/account/3f4a5903-b387-41b5-b451-5a982541b3f9/workspace/4f1de454-4acf-4db1-99c0-9a7ba9a6db9c/flow-runs/flow-run/eab72eaa-f6cf-4959-9aa9-7bdd5eae1fb9

11:36:56.002 | INFO    | Flow run 'magenta-camel' - Created task run 'launch_script_on_perlmutter-0' for task 'launch_script_on_perlmutter'

11:36:56.005 | INFO    | Flow run 'magenta-camel' - Executing 'launch_script_on_perlmutter-0' immediately...

11:38:03.269 | INFO    | Task run 'launch_script_on_perlmutter-0' - Finished in state Completed()

...

11:40:45.372 | INFO    | Flow run 'magenta-camel' - Created task run 'launch_script_on_perlmutter-3' for task 'launch_script_on_perlmutter'

11:40:45.376 | INFO    | Flow run 'magenta-camel' - Executing 'launch_script_on_perlmutter-3' immediately...

11:41:50.864 | INFO    | Task run 'launch_script_on_perlmutter-3' - Finished in state Completed()

11:41:51.089 | INFO    | Flow run 'magenta-camel' - Finished in state Completed('All states completed.')

[Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`'))]